# Test of Piecewise Smooth Quadratic Estimators

In [1]:
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
import interval as ival
import psl_bounds as pl
import psqe_bounds as pq

Auxiliary plot routine

In [2]:
def plot_problem(flist, minlist, a, b):
    colors = ['r-', 'b-', 'g-', 'y-', 'm-', 'c-']
    step = (b - a) / 1000.
    ta = np.arange(a, b + step, step)
    num_points = len(ta)
    num_funcs = len(flist)

    fta = np.empty([num_funcs, num_points])
    print(fta)

    for i in range(num_funcs):
        for j in range(num_points):
            fta[i, j] = flist[i](ta[j])
    lb = np.amin(fta)
    ub = np.amax(fta)
    d = (ub - lb) * 0.1
    for i in range(num_funcs):
        plt.plot(ta, fta[i, :], colors[i])
    for point in min_list:
        plt.scatter(point[0], point[1], s=20)
    plt.ylim([lb - d, ub + d])
    plt.show()

Test problem

In [3]:
x = sym.symbols('x')
e_f = x ** 3 + 2 * x ** 2 - 4 * x + 1
e_nf = -e_f
e_df = sym.diff(e_f)
e_ndf = -e_df
e_ddf = sym.diff(e_df)
print(e_f, e_df, e_ddf)
f = sym.lambdify(x, e_f)
nf = sym.lambdify(x, e_nf)
df = sym.lambdify(x, e_df)
ndf = sym.lambdify(x, e_ndf)
ddf = sym.lambdify(x, e_ddf)
print(f(2.), df(2.))
a = -1.
b = 2.
idef = ival.Interval([a, b])
di = df(idef)

# print(f(idef), df(idef), ddf(idef))
ddi = ddf(idef)
print(di, ddi)

x**3 + 2*x**2 - 4*x + 1 3*x**2 + 4*x - 4 6*x + 4
9.0 16.0
[-8.0, 16.0] [-2.0, 16.0]


In [4]:

# estim = pq.PSQE_Under(a, b, ddi.x[0], ddi.x[1], f, df)
l = max(-di.x[0], di.x[1])
L = max(-ddi.x[0], ddi.x[1])
# print(L)
estim_pl = pl.PSL_Bounds(a, b, di.x[0], di.x[1], f, True)
estim_plip = pl.PSL_Bounds(a, b, -l, l, f, True)
estim_pl_ob = pl.PSL_Bounds(a, b, di.x[0], di.x[1], f, False)

estim_int = pq.PSQE_Bounds(a, b, ddi.x[0], ddi.x[1], f, df, True)
estim_lip = pq.PSQE_Bounds(a, b, -L, L, f, df, True)
estim_int_ob = pq.PSQE_Bounds(a, b, ddi.x[0], ddi.x[1], f, df, False)

print(estim_pl)
print(estim_plip)
print(estim_int)
print(estim_lip)
print(estim_lip.lower_bound_and_point())
min_list = [estim_pl.lower_bound_and_point(), estim_plip.lower_bound_and_point(), estim_int.lower_bound_and_point(),
            estim_lip.lower_bound_and_point()]
print(min_list)

Piecewise linear estimator a = -1.0, b = 2.0, c = 0.875, alp = -8.0, bet = 16.0, fa = 6.0, fb = 9.0
Piecewise linear estimator a = -1.0, b = 2.0, c = 0.40625, alp = -16.0, bet = 16.0, fa = 6.0, fb = 9.0
Estimator a = -1.0, b = 2.0, c = 0.25, d = 1.75, alp = -2.0, bet = 16.0, fa = 6.0, fb = 9.0, dfa = -5.0, dfb = 16.0
Estimator a = -1.0, b = 2.0, c = -0.38247282608695654, d = 1.7737771739130435, alp = -16.0, bet = 16.0, fa = 6.0, fb = 9.0, dfa = -5.0, dfb = 16.0
(0.5475543478260869, -7.057958707466919)
[(0.875, -9.0), (0.40625, -16.5), (0.71875, -3.5703125), (0.5475543478260869, -7.057958707466919)]


Vizualize test problem

In [5]:
plot_problem([f, estim_int_ob, estim_int.estimator, estim_pl, estim_pl_ob], min_list, a, b)

[[0.00000000e+000 2.16054642e-314 2.69384353e-319 ... 4.96844977e+180
  2.42834334e+198 1.22681370e-259]
 [1.33629952e-152 2.57026717e+151 2.47935593e-091 ... 8.87693540e+252
  2.41317562e+185 2.18174346e+243]
 [5.98150244e-154 2.54723493e+151 9.31729203e+242 ... 5.48369079e+247
  1.44421179e+214 8.76393933e+252]
 [2.44077856e-154 1.67761111e+150 3.18489772e-114 ... 3.39066966e-009
  1.63154555e-086 8.06274510e+000]
 [3.44981361e+175 1.81786144e+185 4.35604441e-114 ... 2.17237419e-153
  2.25064918e-153 2.18183985e+243]]


TypeError: 'PSQE_Bounds' object is not callable